In [ ]:
# Install dependencies

In [ ]:
!pip install scikit-surprise --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# !pip install kaggle --quite

In [ ]:
!pip install dask --quiet

In [ ]:
!pip install Dataset dataset --quiet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [38]:
from surprise import SVD, NMF, KNNBasic, accuracy
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV

In [ ]:
# Import files

In [ ]:
import pandas as pd
# use rating_complete first, then animelist
# Surprise expects : user, item, ratings

# anime_df = pd.read_csv("/content/drive/Othercomputers/My MacBook Air/Reccomender_Systems/Kaggle files/anime.csv")
# # anime_with_syn = pd.read_csv("/content/drive/Othercomputers/My MacBook Air/Reccomender_Systems/Kaggle files/anime_with_synopsis.csv")
# animelist_df = pd.read_csv("/content/drive/Othercomputers/My MacBook Air/Reccomender_Systems/Kaggle files/animelist.csv")
ratings_df = pd.read_csv("/content/drive/Othercomputers/My MacBook Air/Reccomender_Systems/Kaggle files/rating_complete.csv", usecols=range(3), header=0, names=('user', 'item', 'rating'), nrows=500000)

In [ ]:
anime_df.head(1)

In [ ]:
# anime_with_syn.head() # not needed

In [28]:
animelist_df.head(1)

,user_id,anime_id,rating,watching_status,watched_episodes
0,0,67,9,1,1


In [29]:
ratings_df.head(1)

,user,item,rating
0,0,430,9


In [ ]:
# Using the Surprise library with the different
# The lower the RMSE / MSE the better

In [30]:
from surprise import Dataset, Reader

# Prepare the dataset for Surprise
reader = Reader(rating_scale=(1, 10))

data = Dataset.load_from_df(ratings_df[['user', 'item', 'rating']], reader)

In [31]:
# Use SVD algorithm
algo = SVD()

# Split the data into training and test set
trainset, testset = train_test_split(data, test_size=0.25)

# Train the algorithm using the trainset
algo.fit(trainset)

# Predict ratings
predictions = algo.test(testset)

# Compute the Root Mean Squared Error and Mean Squared Error
accuracy.rmse(predictions)
accuracy.mse(predictions)

RMSE: 1.2331
MSE: 1.5206


1.5206381921896646

In [32]:
# Use the KNNBasic algorithm
algo = KNNBasic()

# Train the algorithm on the trainset
algo.fit(trainset)

# Predict ratings
predictions = algo.test(testset)

# Compute the Root Mean Squared Error and Mean Squared Error
accuracy.rmse(predictions)
accuracy.mse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.3598
MSE: 1.8491


1.8491366018012765

In [33]:
# Use the NMF algorithm
algo = NMF()

# Train the algorithm on the trainset
algo.fit(trainset)

# Predict ratings
predictions = algo.test(testset)

# Compute the Root Mean Squared Error and Mean Squared Error
accuracy.rmse(predictions)
accuracy.mse(predictions)

RMSE: 2.1210
MSE: 4.4985


4.498501411313473

In [ ]:
# Cross Validation and Grid Search

In [35]:
# CROSS VALIDATION

# Parameter grid
param_grid = {
    'n_epochs': [5, 10, 20, 30],
    'lr_all': [0.002, 0.003, 0.005],
    'reg_all': [0.4, 0.6]
}

# Perform grid search
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mse'], cv=5)
gs.fit(data)

# Best RMSE score
print(gs.best_score['rmse'])

# Lists the params with the best RMSE score
print(gs.best_params['rmse'])

1.3011919757945123
{'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.4}


In [36]:
# Best MSE score
print("Best MSE score:", gs.best_score['mse'])

# Lists the params with the best MSE score
print("Best parameters for MSE:", gs.best_params['mse'])

Best MSE score: 1.6931094456899298
Best parameters for MSE: {'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.4}


In [39]:
# Use the NMF algorithm
algo = NMF()

# Perform cross validation
results = cross_validate(algo, data, measures=['rmse', 'mse'], cv=5, verbose=True)

Evaluating RMSE, MSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    2.1245  2.1170  2.1153  2.1172  2.1288  2.1206  0.0052  
MSE (testset)     4.5137  4.4819  4.4743  4.4827  4.5320  4.4969  0.0221  
Fit time          13.71   14.98   14.90   16.69   15.99   15.25   1.02    
Test time         0.48    1.44    1.25    1.25    1.31    1.15    0.34    


In [40]:
# List of algorithms
algos = [NMF(), SVD(), KNNBasic()]

# Iterate over all algorithms
for algo in algos:
    # Perform cross-validation
    results = cross_validate(algo, data, measures=['rmse', 'mse'], cv=5, verbose=True)

    # Display results for each algorithm
    print("Algorithm:", algo.__class__.__name__)
    print("Average RMSE:", sum(results['test_rmse']) / 5)
    print("Average MSE:", sum(results['test_mse']) / 5)
    print("\n")


Evaluating RMSE, MSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    2.1359  2.1121  2.1235  2.1248  2.1373  2.1267  0.0092  
MSE (testset)     4.5621  4.4611  4.5092  4.5149  4.5680  4.5230  0.0391  
Fit time          14.40   15.41   14.71   15.13   14.83   14.90   0.35    
Test time         1.27    1.29    1.38    1.21    1.21    1.27    0.06    
Algorithm: NMF
Average RMSE: 2.1267254452877906
Average MSE: 4.523045641069475


Evaluating RMSE, MSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2250  1.2318  1.2372  1.2313  1.2336  1.2318  0.0040  
MSE (testset)     1.5006  1.5174  1.5306  1.5161  1.5217  1.5173  0.0098  
Fit time          7.33    7.02    7.46    7.31    7.41    7.30    0.15    
Test time         1.42    1.35    1.36    1.39    1.41    1.39    0.03    
Algorithm: SVD
Average RMSE: 1.2317698917788529
Average MSE: 